<a href="https://www.kaggle.com/code/dilekdd/rule-based-classification-project?scriptVersionId=194528725" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

A gaming company wants to create new level-based customer definitions (personas) by using certain characteristics of its customers and, based on these new customer definitions, establish segments to estimate how much potential new customers might contribute to the company's revenue on average.

For example: They want to determine the average revenue generated by a 25-year-old male user from Turkey who uses iOS.

The Persona.csv dataset contains the prices of products sold by an international gaming company, along with some demographic information about the users who purchased these products. The dataset is composed of records generated from each sales transaction. This means that the table is not deduplicated. In other words, a user with certain demographic characteristics may have made multiple purchases.

Variables;

PRICE – The amount spent by the customer
SOURCE – The type of device the customer connected from
SEX – The customer's gender
COUNTRY – The customer's country
AGE – The customer's age

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)

Read the persona.csv file and display general information about the data set.

In [2]:
df = pd.read_csv("/kaggle/input/persona-dataset/persona.csv")
df.columns
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   PRICE    5000 non-null   int64 
 1   SOURCE   5000 non-null   object
 2   SEX      5000 non-null   object
 3   COUNTRY  5000 non-null   object
 4   AGE      5000 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 234.4+ KB


In [3]:
def quick_overview(df_summary):
    print(f"First 5 observations\n{df_summary.head()}\n")
    print(f"Missing values by variables\n{df_summary.isnull().sum()}\n")
    print(f"Descriptive statistics for numerical variables\n{df_summary.describe()}\n")
    print(f"Overview of the DataFrame :")
    df_summary.info()

quick_overview(df)

First 5 observations
   PRICE   SOURCE   SEX COUNTRY  AGE
0     39  android  male     bra   17
1     39  android  male     bra   17
2     49  android  male     bra   17
3     29  android  male     tur   17
4     49  android  male     tur   17

Missing values by variables
PRICE      0
SOURCE     0
SEX        0
COUNTRY    0
AGE        0
dtype: int64

Descriptive statistics for numerical variables
             PRICE          AGE
count  5000.000000  5000.000000
mean     34.132000    23.581400
std      12.464897     8.995908
min       9.000000    15.000000
25%      29.000000    17.000000
50%      39.000000    21.000000
75%      39.000000    27.000000
max      59.000000    66.000000

Overview of the DataFrame :
<class 'pandas.core.frame.DataFrame'>
Index: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   PRICE    5000 non-null   int64 
 1   SOURCE   5000 non-null   object
 2   SEX      5000 non-null   object

How many unique SOURCE are there? What are their frequencies?

In [4]:
df["SOURCE"].nunique()
df["SOURCE"].value_counts()

SOURCE
android    2974
ios        2026
Name: count, dtype: int64

How many unique PRICEs are there?

In [5]:
df["PRICE"].nunique()

6

Sales amount by PRICE?

In [6]:
df["PRICE"].value_counts()

PRICE
29    1305
39    1260
49    1031
19     992
59     212
9      200
Name: count, dtype: int64

Sales count by COUNTRY?

In [7]:
df["COUNTRY"].value_counts()

COUNTRY
usa    2065
bra    1496
deu     455
tur     451
fra     303
can     230
Name: count, dtype: int64

Total income by country

In [8]:
df.groupby('COUNTRY')['PRICE'].sum()

COUNTRY
bra    51354
can     7730
deu    15485
fra    10177
tur    15689
usa    70225
Name: PRICE, dtype: int64

Sales numbers by SOURCE

In [9]:
df.groupby("SOURCE").size()

SOURCE
android    2974
ios        2026
dtype: int64

Average PRICE by country

In [10]:
df.groupby("COUNTRY").agg({"PRICE": "mean"})

,PRICE
COUNTRY,
bra,34.327540
can,33.608696
deu,34.032967
fra,33.587459
tur,34.787140
usa,34.007264


 Average PRICE by SOURCE type

In [11]:
df.groupby("SOURCE").agg({"PRICE": "mean"})

,PRICE
SOURCE,
android,34.174849
ios,34.069102


Average PRICE by COUNTRY and SOURCE breakdown

In [12]:
df.groupby(["COUNTRY", "SOURCE"]).agg({"PRICE": "mean"})

PRICE
COUNTRY SOURCE            
bra     android  34.387029
        ios      34.222222
can     android  33.330709
        ios      33.951456
deu     android  33.869888
        ios      34.268817
fra     android  34.312500
        ios      32.776224
tur     android  36.229437
        ios      33.272727
usa     android  33.760357
        ios      34.371703

Average revenue by COUNTRY, SOURCE, SEX, AGE breakdown

In [13]:
new_df = df.groupby(["COUNTRY", "SOURCE", "SEX", "AGE"]).agg({"PRICE": "mean"})
new_df.head()

PRICE
COUNTRY SOURCE  SEX    AGE           
bra     android female 15   38.714286
                       16   35.944444
                       17   35.666667
                       18   32.255814
                       19   35.206897

Sort the output by PRICE

In [14]:
agg_df = new_df.sort_values(by="PRICE", ascending=False, ignore_index=True)
agg_df.head(5)

,PRICE
0,59.0
1,59.0
2,59.0
3,54.0
4,49.0


Convert the names in the index to variable names

In [15]:
agg_df = new_df.sort_values(by="PRICE", ascending=False).reset_index()
agg_df.head(5)

,COUNTRY,SOURCE,SEX,AGE,PRICE
0,bra,android,male,46,59.0
1,usa,android,male,36,59.0
2,fra,android,female,24,59.0
3,usa,ios,male,32,54.0
4,deu,android,female,36,49.0


Convert the Age variable to a categorical variable and add it to agg_df

In [16]:
agg_df["AGE_CAT"] = pd.cut(agg_df["AGE"], bins = [0, 18, 23, 30, 40, 70], labels = ['0_18', '19_23', '24_30', '31_40', '41_70'])
agg_df.head(5)

,COUNTRY,SOURCE,SEX,AGE,PRICE,AGE_CAT
0,bra,android,male,46,59.0,41_70
1,usa,android,male,36,59.0,31_40
2,fra,android,female,24,59.0,24_30
3,usa,ios,male,32,54.0,31_40
4,deu,android,female,36,49.0,31_40


Define new level-based customers (personas) and add them to the data set as variables.
The name of the new variable to be added: customers_level_based
You need to create the customers_level_based variable by bringing together the observations in the output you will obtain in the previous question.

In [17]:
agg_df['customers_level_based'] = agg_df['COUNTRY'].str.upper() + '_' + agg_df['SOURCE'].str.upper() + '_' + agg_df['SEX'].str.upper() + '_' + agg_df['AGE_CAT'].astype(str)


In [18]:
agg_df = agg_df[['customers_level_based', 'PRICE']]
agg_df.head()

,customers_level_based,PRICE
0,BRA_ANDROID_MALE_41_70,59.0
1,USA_ANDROID_MALE_31_40,59.0
2,FRA_ANDROID_FEMALE_24_30,59.0
3,USA_IOS_MALE_31_40,54.0
4,DEU_ANDROID_FEMALE_31_40,49.0


Separate new customers (Example: USA_ANDROID_MALE_0_18) into 4 segments according to PRICE.
Add the segments to agg_df as variables with the name SEGMENT.
Describe the segments (Group by according to the segments and get the price mean, max, sum).

In [19]:
agg_df["SEGMENT"] = pd.qcut(agg_df["PRICE"], 4, labels=["D", "C", "B", "A"])
print(agg_df.head(5))
print(agg_df.tail(5))

      customers_level_based  PRICE SEGMENT
0    BRA_ANDROID_MALE_41_70   59.0       A
1    USA_ANDROID_MALE_31_40   59.0       A
2  FRA_ANDROID_FEMALE_24_30   59.0       A
3        USA_IOS_MALE_31_40   54.0       A
4  DEU_ANDROID_FEMALE_31_40   49.0       A
        customers_level_based  PRICE SEGMENT
343      USA_IOS_FEMALE_31_40   19.0       D
344      USA_IOS_FEMALE_24_30   19.0       D
345  CAN_ANDROID_FEMALE_24_30   19.0       D
346     FRA_ANDROID_MALE_0_18   19.0       D
347    DEU_ANDROID_MALE_24_30    9.0       D


In [20]:
segment_analysis = agg_df.groupby("SEGMENT", observed=False).agg({"PRICE": ["mean", "max", "sum"]})
print(segment_analysis)

             PRICE                        
              mean        max          sum
SEGMENT                                   
D        27.302596  31.105263  2375.325850
C        32.933339  34.000000  3128.667165
B        35.436170  37.000000  2870.329792
A        41.434736  59.000000  3521.952577


Which segment does a 33-year-old Turkish woman using ANDROID belong to and how much income is expected to be earned on average?

Which segment does a 35-year-old French woman using IOS belong to and how much income is expected to be earned on average?

In [21]:
new_user = "TUR_ANDROID_FEMALE_31_40"
new_user_segment = agg_df[agg_df["customers_level_based"] == new_user]
print(f"33 year-old Turkish female who uses ANDROID belongs to the segment {new_user_segment['SEGMENT'].values[0]} and the average income: {new_user_segment['PRICE'].values[0]}")

33 year-old Turkish female who uses ANDROID belongs to the segment A and the average income: 43.0


In [22]:
new_user = "FRA_IOS_FEMALE_31_40"
french_female_segment = agg_df[agg_df["customers_level_based"] == new_user]
print(f"35 year-old French female who uses ANDROID belongs to the segment {french_female_segment['SEGMENT'].values[0]} and the average income: {french_female_segment['PRICE'].values[0]}")

35 year-old French female who uses ANDROID belongs to the segment C and the average income: 33.0
